In [ ]:
#######################################################################################################################
# Project: Deep Virtual Rapport Agent (data preprocessing)
#
#     Jan Ondras (jo951030@gmail.com)
#     Institute for Creative Technologies, University of Southern California
#     April-October 2019
#
#######################################################################################################################
# Generate segmented/sequenced test dataset from ccdb dataset
#
#     For head gestures nod, shake, and tilt.
#
#     So far only for the basic set of the first 8 sessions.
#
#     Run after the annotate_features.ipynb script was run.
#
#     Input features: dvra_datasets/ccdb/annotated_features
#     Output dataset: dvra_datasets/ccdb/segmented_datasets
#
#     The generated dataset was used for cross-dataset testing of the developed Head Gesture Detector.
#     In future, the ccdb dataset can be included in the training data, extending the 4comb dataset to 5comb.
#######################################################################################################################

In [1]:
###########################################################
import numpy as np
random_seed = 37
np.random.seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)
###########################################################

import os
import glob
import time
import warnings
import pandas as pd
from collections import defaultdict


# Mask value (if all features for a given sample timestep are equal to MASK_VALUE, 
# then the sample timestep will be masked (skipped))
# Cannot use MASK_VALUE=0.0, as it corresponds to no movement (derivatives are zero)
# Cannot use MASK_VALUE=np.inf, as RandomUnderSampler cannot handle NaNs and Inf values
MASK_VALUE = 7777777.7777777
    
    
def generate_dataset(selected_features, window_size, val_size, head_gesture):
    '''
    Split dataset (csv files of recordings) into train/val paritions (to train final model to be used for cross-dataset prediction).
    Also prepare test partition that contains all the data for cross-dataset testing. 
    Segment both partitions of the dataset.
        Dataset is segmented into same-length (window_size) sequences.
        Feature segments are pre-padded with MASK_VALUE-s and label segments with 0 (not a nod/shake/tilt).
    For other datasets need to modify to include all available annotations (for nod, shake, tilt).
    One output file is saved. 
    ''' 
    
    dataset_output_filename_prefix = f'/home/ICT2000/jondras/dvra_datasets/ccdb/segmented_datasets/'
    
    dataset_type = f'{window_size}ws_{len(selected_features)}f'
    if not os.path.exists(dataset_output_filename_prefix):
        os.makedirs(dataset_output_filename_prefix)
    
    print(f'Head gesture: {head_gesture}')
    print(f'Window size: {window_size}')
    n_features = len(selected_features)
    print(f'Selected features: \n\t{selected_features}')
        
    
    def get_segments(df):
        '''
        Generate segments (X (features) and Y (labels)) from the dataframe. 
        
        Returns 2 lists of 2D arrays.
        '''
        
        X = []
        Y = []

        # Pre-pad all features and labels with (window_size - 1) MASK_VALUE-s 
        padded_features = np.pad(df.values[:, :-1], ((window_size - 1, 0), (0, 0)), 
                                 mode='constant', constant_values=(MASK_VALUE, MASK_VALUE))
        # Labels are padded with 0 mask value (indicating not a nod)
        padded_labels   = np.pad(df.values[:, -1],  (window_size - 1, 0), 
                                 mode='constant', constant_values=(0, 0))
        
        assert padded_features.shape[1] == n_features
        assert padded_labels.shape[0] == padded_features.shape[0]
        assert len(padded_features) - window_size + 1 == len(df), 'Padding failed!'

        # Slide window of length window_size over the padded features/labels
        for i in range(len(df)):       
            X.append( padded_features[i:i + window_size] )
            Y.append( padded_labels[i:i + window_size] )

        return X, Y
    
    
    # Load the annotated feature files
    input_annotated_features_dir = '/home/ICT2000/jondras/dvra_datasets/ccdb/annotated_features'
    input_filenames = np.array(sorted(glob.glob(input_annotated_features_dir + '/*.csv')))
    n_subjects = len(input_filenames)
    print(f'\t {n_subjects} subjects/sessions')
    
    start_time = time.time()
    segments = defaultdict(list)
    for annotated_features_file in input_filenames:
        # Take only selected features and annotation columns.
        df = pd.read_csv(annotated_features_file)[selected_features + [head_gesture]]
        
        # Get all segments for cross-dataset testing
        X_test, Y_test = get_segments(df=df)
        assert len(X_test) == len(df)
        segments['X_test'].extend(X_test)
        segments['Y_test'].extend(Y_test)
        segments['test_len'].append(len(X_test))

    # Convert lists to numpy arrays and reshape Y to be 3D (as needed for training)
    for key in segments.keys():
        segments[key] = np.array(segments[key])
        if key[0] == 'Y':
            segments[key] = np.expand_dims(segments[key], axis=-1)               
        print(key, segments[key].shape)

    # Save test segmented data for this fold
    segments['selected_features'] = selected_features
    segments['WINDOW_SIZE'] = window_size
    segments['MASK_VALUE'] = MASK_VALUE
    np.savez(dataset_output_filename_prefix + f'ccdb_{head_gesture}_{dataset_type}', **segments)

    print(f'\t\t Number of test examples per class: \t{np.unique(segments[f"Y_test"][:, -1], return_counts=True)}')        
    print(f'\t\t Total time taken: {time.time() - start_time} s')
    print('====================================================================================================')


In [2]:
# selected_features_1 = [
#     'diff_ pose_Tx', 
#     'diff_ pose_Ty', 
#     'diff_ pose_Tz',

#     'diff_ pose_Rx', 
#     'diff_ pose_Ry', 
#     'diff_ pose_Rz',
# ]
selected_features_2 = [
    'diff_ pose_Tx', 
    'diff_ pose_Ty', 
    'diff_ pose_Tz',
    
    'diff2_ pose_Tx', 
    'diff2_ pose_Ty', 
    'diff2_ pose_Tz',

    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',
    
    'diff2_ pose_Rx', 
    'diff2_ pose_Ry', 
    'diff2_ pose_Rz',
]

for sf in [selected_features_2]:
    for ws in [32]:
        for hg in ['nod', 'shake', 'tilt']:
            generate_dataset(selected_features=sf, window_size=ws, val_size=0.15, head_gesture=hg)

Head gesture: nod
Window size: 32
Selected features: 
	['diff_ pose_Tx', 'diff_ pose_Ty', 'diff_ pose_Tz', 'diff2_ pose_Tx', 'diff2_ pose_Ty', 'diff2_ pose_Tz', 'diff_ pose_Rx', 'diff_ pose_Ry', 'diff_ pose_Rz', 'diff2_ pose_Rx', 'diff2_ pose_Ry', 'diff2_ pose_Rz']
	 16 subjects/sessions
X_test (142442, 32, 12)
Y_test (142442, 32, 1)
test_len (16,)
		 Number of test examples per class: 	(array([0., 1.]), array([135983,   6459]))
		 Total time taken: 24.609424829483032 s
Head gesture: shake
Window size: 32
Selected features: 
	['diff_ pose_Tx', 'diff_ pose_Ty', 'diff_ pose_Tz', 'diff2_ pose_Tx', 'diff2_ pose_Ty', 'diff2_ pose_Tz', 'diff_ pose_Rx', 'diff_ pose_Ry', 'diff_ pose_Rz', 'diff2_ pose_Rx', 'diff2_ pose_Ry', 'diff2_ pose_Rz']
	 16 subjects/sessions
X_test (142442, 32, 12)
Y_test (142442, 32, 1)
test_len (16,)
		 Number of test examples per class: 	(array([0., 1.]), array([140519,   1923]))
		 Total time taken: 24.8432035446167 s
Head gesture: tilt
Window size: 32
Selected featur